In [1]:
import matplotlib.pyplot as plt
from nltk.corpus import wordnet as wn
from gensim.models.poincare import PoincareModel
from sklearn.model_selection import train_test_split
import numpy as np
import nltk
import matplotlib.pyplot as plt
import numpy as np
import nltk
from nltk.corpus import wordnet as wn
from gensim.models.poincare import PoincareModel
from sklearn.model_selection import train_test_split
import random
import random
import numpy as np
import pandas as pd
from collections import defaultdict

from nltk.corpus import wordnet as wn
from gensim.models.poincare import PoincareModel
from sklearn.model_selection import train_test_split

In [2]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\yoel1\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [1]:
import os
import random
import pandas as pd
import numpy as np
from collections import defaultdict
from sklearn.model_selection import train_test_split

import nltk
from nltk.corpus import wordnet as wn


import autograd
from autograd import grad
import autograd.numpy as anp  
from types import MethodType

from gensim.models.poincare import (
    PoincareKeyedVectors,
    PoincareModel,
    PoincareBatch
)


def no_op_compute_all(self):

    self.loss = 0.0


PoincareBatch.compute_all = no_op_compute_all



def anp_safe_norm(x, axis=None, eps=1e-9):

    return anp.sqrt(anp.sum(x * x, axis=axis) + eps)

def anp_safe_arccosh(x, eps=1e-9):

    clipped_x = anp.clip(x, 1.0 + eps, 1e15)
    inside = clipped_x * clipped_x - 1.0
    return anp.log(clipped_x + anp.sqrt(inside + eps))


def custom_loss_with_alpha(matrix, alpha, regularization_coeff=1.0):

    kappa = -anp.exp(alpha) 
    vector_u = matrix[0]
    vectors_v = matrix[1:]  

    
    eucl_dists = anp_safe_norm(vector_u - vectors_v, axis=1)
    norm_u = anp_safe_norm(vector_u)
    norms_v = anp_safe_norm(vectors_v, axis=1)

    denom_u = (1.0 + kappa * (norm_u**2))
    denom_v = (1.0 + kappa * (norms_v**2))
    denom = denom_u * denom_v
    denom = anp.clip(denom, 1e-9, 1e15)

    gamma = 1.0 + 2.0 * (eucl_dists**2) / denom
    gamma = anp.clip(gamma, 1.0 + 1e-9, 1e15)

    poincare_dists = anp_safe_arccosh(gamma)
    exp_neg = anp.exp(-poincare_dists)

    reg_term = regularization_coeff * anp.sum(vectors_v[0] * vectors_v[0])
    return -anp.log(exp_neg[0] / anp.sum(exp_neg)) + reg_term

_loss_grad_wrt_both = grad(custom_loss_with_alpha, argnum=[0, 1])



def monkey_patch_train_on_batch_with_alpha(self, relations, check_gradients=False):

    all_negatives = self._sample_negatives_batch(r[0] for r in relations)
    batch = self._prepare_training_batch(relations, all_negatives, check_gradients)

    total_loss = 0.0
    for (u, v), negs in zip(relations, all_negatives):
        vec_u = self.kv.vectors[u]
        vec_v = self.kv.vectors[v]
        vec_negs = self.kv.vectors[negs]

        sub_matrix = np.vstack((vec_u, vec_v, vec_negs))

        grad_matrix, grad_alpha = _loss_grad_wrt_both(
            sub_matrix, self.alpha, self.regularization_coeff
        )

        self.kv.vectors[u] -= self.alpha_emb * grad_matrix[0]
        self.kv.vectors[v] -= self.alpha_emb * grad_matrix[1]
        for i, neg_idx in enumerate(negs):
            self.kv.vectors[neg_idx] -= self.alpha_emb * grad_matrix[2 + i]

        self.kv.vectors[u] = self._clip_vectors(self.kv.vectors[u], self.epsilon)
        self.kv.vectors[v] = self._clip_vectors(self.kv.vectors[v], self.epsilon)
        self.kv.vectors[negs] = self._clip_vectors(self.kv.vectors[negs], self.epsilon)

        self.alpha -= self.lr_alpha * grad_alpha

        example_loss = float(custom_loss_with_alpha(sub_matrix, self.alpha, self.regularization_coeff))
        total_loss += example_loss

    batch.loss = total_loss / len(relations)
    return batch



def set_learnable_kappa_reparam(model, alpha_init=0.0, lr_alpha=1e-3, alpha_emb=0.1):
    model.alpha = alpha_init
    model.lr_alpha = lr_alpha
    model.alpha_emb = alpha_emb
    from types import MethodType
    model._train_on_batch = MethodType(monkey_patch_train_on_batch_with_alpha, model)

    print(f"[INFO] kappa reparam => alpha_init={alpha_init}, lr_alpha={lr_alpha}, alpha_emb={alpha_emb}")



def curved_poincare_distance(u, v, alpha):

    kappa = -np.exp(alpha)
    eucl_dist = np.linalg.norm(u - v)
    norm_u = np.linalg.norm(u)
    norm_v = np.linalg.norm(v)

    denom = (1.0 + kappa*norm_u**2) * (1.0 + kappa*norm_v**2)
    if denom <= 1e-15:
        denom = 1e-15
    gamma = 1.0 + 2.0*(eucl_dist**2)/denom
    if gamma < 1.0:
        gamma = 1.0
    return np.arccosh(gamma)

def custom_vector_distance(self, u, v):
    alpha = getattr(self, "model_alpha", 0.0)
    return curved_poincare_distance(u, v, alpha)

def custom_vector_distance_batch(self, u, all_v):
    alpha = getattr(self, "model_alpha", 0.0)
    # Vectorized
    kappa = -np.exp(alpha)
    euc_dists = np.linalg.norm(u - all_v, axis=1)
    norm_u = np.linalg.norm(u)
    norms_v = np.linalg.norm(all_v, axis=1)
    denom = (1.0 + kappa*norm_u**2)*(1.0 + kappa*(norms_v**2))
    denom[denom < 1e-15] = 1e-15
    gamma = 1.0 + 2.0*(euc_dists**2)/denom
    gamma[gamma < 1.0] = 1.0
    return np.arccosh(gamma)

PoincareKeyedVectors.vector_distance = custom_vector_distance
PoincareKeyedVectors.vector_distance_batch = custom_vector_distance_batch

def no_op_loss_fn(*args, **kwargs):
    return 0.0

PoincareModel._loss_fn = no_op_loss_fn 


def prepare_mammal_data(subset_size=10000, test_size=0.2, seed=42):

    def extract_transitive_hypernyms_mammals():
        mammal_synset = wn.synset('mammal.n.01')

        mammal_synsets = set(mammal_synset.closure(lambda s: s.hyponyms()))
        mammal_synsets.add(mammal_synset)

        relations = []
        for syn in mammal_synsets:
            paths = syn.hypernym_paths() 
            for path in paths:
                for ancestor in path:
                    if ancestor in mammal_synsets and ancestor != syn:
                        relations.append((syn.name(), ancestor.name()))
        return relations

    print("Building transitive closure for the Mammal subtree...")
    relations_list = extract_transitive_hypernyms_mammals()
    print(f"Total transitive hypernym pairs (mammal subtree): {len(relations_list)}")

    # Optional Subsample
    if subset_size is not None and len(relations_list) > subset_size:
        random.seed(seed)
        relations_list = random.sample(relations_list, subset_size)
        print(f"Using a SUBSET of {subset_size} edges for faster testing.")

    # Train/Test Split
    train_relations, test_relations = train_test_split(
        relations_list, test_size=test_size, random_state=seed
    )
    print(f"Train relations: {len(train_relations)}")
    print(f"Test relations : {len(test_relations)}")

    combined_relations = train_relations + test_relations
    return train_relations, test_relations, combined_relations


def flip_edge_directions(relations, flip_probability=0.1, seed=42):
    """Flips each edge (u->v) to (v->u) with probability flip_probability."""
    random.seed(seed)
    flipped = []
    for (child, ancestor) in relations:
        if random.random() < flip_probability:
            flipped.append((ancestor, child))
        else:
            flipped.append((child, ancestor))
    return flipped


def train_and_evaluate_poincare(
    train_relations,
    test_relations,
    combined_relations,
    embedding_dim=5,
    epochs=300,
    n_negatives_strict=500,
):

    from gensim.models.poincare import PoincareModel

    print(f"[INFO] Training Poincaré (dim={embedding_dim}, epochs={epochs}) ...")

    model = PoincareModel(
        train_data=train_relations,
        size=embedding_dim,
        negative=10,
        burn_in=10
    )
    
    set_learnable_kappa_reparam(model, alpha_init=0.0, lr_alpha=1e-3, alpha_emb=0.1)

    
    model.train(epochs=epochs)

    
    final_alpha = model.alpha
    final_kappa = -np.exp(final_alpha)
    print(f"[INFO] Final alpha={final_alpha}, kappa={final_kappa}")
    model.kv.model_alpha = final_alpha  

    
    from collections import defaultdict
    u_to_all_neighbors = defaultdict(set)
    for (c, a) in combined_relations:
        u_to_all_neighbors[c].add(a)

    vocab_nodes = set(model.kv.index_to_key)
    vocab_list = list(vocab_nodes)
    all_edges_set = set(combined_relations)

    
    recon_mr_strict = reconstruction_mean_rank_strict_sampled(
        model, train_relations, u_to_all_neighbors, vocab_list, n_negatives_strict
    )
    recon_map_strict = reconstruction_map_strict_sampled(
        model, train_relations, u_to_all_neighbors, vocab_list, n_negatives_strict
    )

    
    lp_mr = link_prediction_mean_rank(model, test_relations, vocab_list, all_edges_set)
    lp_map_ = link_prediction_map(model, test_relations, vocab_list, all_edges_set)
    lp_p10 = precision_at_k(model, test_relations, vocab_list, all_edges_set, k=10)
    lp_r10 = recall_at_k(model, test_relations, vocab_list, all_edges_set, k=10)

    return {
        "reconstruction_mean_rank_strict": recon_mr_strict,
        "reconstruction_map_strict": recon_map_strict,
        "lp_mean_rank": lp_mr,
        "lp_map": lp_map_,
        "lp_precision_10": lp_p10,
        "lp_recall_10": lp_r10
    }



def reconstruction_mean_rank_strict_sampled(
    model, edges, u_to_all_neighbors, vocab_list, n_negatives=500, seed=1234
):
    rng = np.random.default_rng(seed)
    ranks = []
    for (u, v) in edges:
        if (u not in model.kv) or (v not in model.kv):
            continue
        neighbors = u_to_all_neighbors[u]
        neg_candidates = []
        attempts = 0
        while len(neg_candidates) < n_negatives and attempts < 10000:
            candidate = rng.choice(vocab_list)
            if candidate not in (u, v) and (candidate not in neighbors):
                neg_candidates.append(candidate)
            attempts += 1
        candidates = neg_candidates + [v]
        dists = [(c, model.kv.distance(u, c)) for c in candidates]
        sorted_nodes = [x[0] for x in sorted(dists, key=lambda x: x[1])]
        try:
            rank = sorted_nodes.index(v) + 1
            ranks.append(rank)
        except ValueError:
            pass
    return float(np.mean(ranks)) if ranks else 0.0

def reconstruction_map_strict_sampled(
    model, edges, u_to_all_neighbors, vocab_list, n_negatives=500, seed=1234
):
    rng = np.random.default_rng(seed)
    reciprocal_ranks = []
    for (u, v) in edges:
        if (u not in model.kv) or (v not in model.kv):
            continue
        neighbors = u_to_all_neighbors[u]
        neg_candidates = []
        attempts = 0
        while len(neg_candidates) < n_negatives and attempts < 10000:
            candidate = rng.choice(vocab_list)
            if candidate not in (u, v) and (candidate not in neighbors):
                neg_candidates.append(candidate)
            attempts += 1
        candidates = neg_candidates + [v]
        dists = [(c, model.kv.distance(u, c)) for c in candidates]
        sorted_nodes = [x[0] for x in sorted(dists, key=lambda x: x[1])]
        try:
            r = sorted_nodes.index(v) + 1
            reciprocal_ranks.append(1.0 / r)
        except ValueError:
            pass
    return float(np.mean(reciprocal_ranks)) if reciprocal_ranks else 0.0


def link_prediction_mean_rank(
    model, edges, vocab_list, all_edges_set, num_negatives=50
):
    rng = np.random.default_rng(1234)
    ranks = []
    for (source, target) in edges:
        if (source not in model.kv) or (target not in model.kv):
            continue
        neg_candidates = []
        attempts = 0
        while len(neg_candidates) < num_negatives and attempts < 10000:
            candidate = rng.choice(vocab_list)
            # Must not be the true target, nor an existing edge
            if candidate != target and (source, candidate) not in all_edges_set:
                neg_candidates.append(candidate)
            attempts += 1
        if not neg_candidates:
            continue
        candidates = neg_candidates + [target]
        dists = [model.kv.distance(source, c) for c in candidates]
        sorted_candidates = [c for _, c in sorted(zip(dists, candidates), key=lambda x: x[0])]
        rank = sorted_candidates.index(target) + 1
        ranks.append(rank)
    return float(np.mean(ranks)) if ranks else 0.0

def link_prediction_map(model, edges, vocab_list, all_edges_set, num_negatives=50):
    rng = np.random.default_rng(1234)
    reciprocal_ranks = []
    for (source, target) in edges:
        if (source not in model.kv) or (target not in model.kv):
            continue
        neg_candidates = []
        attempts = 0
        while len(neg_candidates) < num_negatives and attempts < 10000:
            candidate = rng.choice(vocab_list)
            if candidate != target and (source, candidate) not in all_edges_set:
                neg_candidates.append(candidate)
            attempts += 1
        if not neg_candidates:
            continue
        candidates = neg_candidates + [target]
        dists = [model.kv.distance(source, c) for c in candidates]
        sorted_candidates = [c for _, c in sorted(zip(dists, candidates), key=lambda x: x[0])]
        rank = sorted_candidates.index(target) + 1
        reciprocal_ranks.append(1.0 / rank)
    return float(np.mean(reciprocal_ranks)) if reciprocal_ranks else 0.0

def precision_at_k(model, edges, vocab_list, all_edges_set, k=10, num_negatives=50):
    rng = np.random.default_rng(1234)
    hits = 0
    count = 0
    for (source, target) in edges:
        if (source not in model.kv) or (target not in model.kv):
            continue
        neg_candidates = []
        attempts = 0
        while len(neg_candidates) < num_negatives and attempts < 10000:
            candidate = rng.choice(vocab_list)
            if candidate != target and (source, candidate) not in all_edges_set:
                neg_candidates.append(candidate)
            attempts += 1
        if not neg_candidates:
            continue
        candidates = neg_candidates + [target]
        dists = [model.kv.distance(source, c) for c in candidates]
        sorted_candidates = [c for _, c in sorted(zip(dists, candidates), key=lambda x: x[0])]
        top_k_nodes = sorted_candidates[:k]
        if target in top_k_nodes:
            hits += 1
        count += 1
    return hits / count if count else 0.0

def recall_at_k(model, edges, vocab_list, all_edges_set, k=10, num_negatives=50):

    return precision_at_k(model, edges, vocab_list, all_edges_set, k, num_negatives)



def run_multiple_experiments(
    address,                
    dim_list,               
    flip_list,              
    subset_size=10000,
    test_size=0.2,
    epochs=300,
    n_negatives_strict=500,
    seed=42
):
   
    train_rel_clean, test_rel, combined_rel = prepare_mammal_data(
        subset_size=subset_size,
        test_size=test_size,
        seed=seed
    )

   
    results_all = []
    for dim in dim_list:
        for flip_probability in flip_list:
            print("========================================================")
            print(f"RUNNING: dim={dim}, flip={flip_probability}")
            print("========================================================")

            
            flipped_train = flip_edge_directions(train_rel_clean, flip_probability, seed=seed)

            
            stats = train_and_evaluate_poincare(
                train_relations=flipped_train,
                test_relations=test_rel,
                combined_relations=combined_rel,
                embedding_dim=dim,
                epochs=epochs,
                n_negatives_strict=n_negatives_strict
            )

            
            row_info = {
                "embedding_dim": dim,
                "flip_probability": flip_probability,
                "subset_size": subset_size,
                "test_size": test_size,
                "epochs": epochs,
                "n_negatives_strict": n_negatives_strict
            }
            row_info.update(stats)
            results_all.append(row_info)

    df = pd.DataFrame(results_all)
    df.to_excel(address, index=False)
    print(f"\nAll experiment results saved to: {address}")
    return df



if __name__ == "__main__":
    dimension_list = [5, 10,20,50,100,200]
    flip_probability_list = [0.0, 0.1, 0.3]
    excel_path = "Y:\\Data Science Readings\\Geometry of Information\\Milestone 3\\Milestone3-Mammals\\MammalsM3Exp.xlsx"

    final_df = run_multiple_experiments(
        address=excel_path,
        dim_list=dimension_list,
        flip_list=flip_probability_list,
        subset_size=10000,   # or smaller
        test_size=0.2,
        epochs=100,
        n_negatives_strict=50,
        seed=42
    )
    print("\nFinal DataFrame of all runs:\n")
    print(final_df)


Building transitive closure for the Mammal subtree...
Total transitive hypernym pairs (mammal subtree): 7051
Train relations: 5640
Test relations : 1411
RUNNING: dim=5, flip=0.0
[INFO] Training Poincaré (dim=5, epochs=100) ...
[INFO] kappa reparam => alpha_init=0.0, lr_alpha=0.001, alpha_emb=0.1
[INFO] Final alpha=-10.186642759436284, kappa=-3.767014227323029e-05
RUNNING: dim=5, flip=0.1
[INFO] Training Poincaré (dim=5, epochs=100) ...
[INFO] kappa reparam => alpha_init=0.0, lr_alpha=0.001, alpha_emb=0.1
[INFO] Final alpha=-0.14581784152425822, kappa=-0.8643151311726412
RUNNING: dim=5, flip=0.3
[INFO] Training Poincaré (dim=5, epochs=100) ...
[INFO] kappa reparam => alpha_init=0.0, lr_alpha=0.001, alpha_emb=0.1
[INFO] Final alpha=-0.3242996127880126, kappa=-0.7230335798173502
RUNNING: dim=10, flip=0.0
[INFO] Training Poincaré (dim=10, epochs=100) ...
[INFO] kappa reparam => alpha_init=0.0, lr_alpha=0.001, alpha_emb=0.1
[INFO] Final alpha=-5.224568299684796, kappa=-0.005382683164429406
